In [1]:
%config IPCompleter.greedy=True

# Defining basic url concatenating function

In [2]:
import requests
import urllib
from bs4 import BeautifulSoup

import pandas as pd
import requests
from io import StringIO
import regex as re
pd.set_option('display.max_columns', None)

In [3]:
def make_url(base_url , comp):
    
    url = base_url
    for r in comp:
        url = '{}/{}'.format(url, r)    
    return url

#  Basic URL and its JSON index

In [4]:
base_url = r"https://www.sec.gov/Archives/edgar/daily-index"
archive_url = r"https://www.sec.gov/Archives"

basic_url = r"https://www.sec.gov/Archives/edgar/daily-index/index.json" #able to parse using requests, bs4 is needed for html

decoded_year_url = requests.get(basic_url).json() 
decoded_year_url

{'directory': {'item': [{'last-modified': '09/03/2014 10:14:30 AM',
    'name': '1994',
    'type': 'dir',
    'href': '1994/',
    'size': '4 KB'},
   {'last-modified': '09/03/2014 10:14:30 AM',
    'name': '1995',
    'type': 'dir',
    'href': '1995/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:11:52 PM',
    'name': '1996',
    'type': 'dir',
    'href': '1996/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:11:56 PM',
    'name': '1997',
    'type': 'dir',
    'href': '1997/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:12:00 PM',
    'name': '1998',
    'type': 'dir',
    'href': '1998/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:12:04 PM',
    'name': '1999',
    'type': 'dir',
    'href': '1999/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:12:09 PM',
    'name': '2000',
    'type': 'dir',
    'href': '2000/',
    'size': '4 KB'},
   {'last-modified': '09/15/2014 05:12:13 PM',
    'name': '2001',
    'type': 'dir

# Going through multiple years and quarters and then outputting all Master Indexes Url that are stored in a list

In [6]:
file_urls = []

yrlist = [2018, 2019]
Qtrlist = [1,]
string="QTR"
output = ["{}{}".format(string, i) for i in Qtrlist]
Qtrlist, output

for x in yrlist:      # for q in output:
    print('-'*100)
    print('Pulling url for Year: {}'.format(x) )
    
    yr_url = make_url(base_url, [x, 'index.json'])
    
    for q in output:  #     for x in yrlist:
                
        year_qtr_url = make_url(base_url, [x, q])
        yr_qtr_url_index = make_url(year_qtr_url, [ 'index.json'])

        print("Year + Quarter URL Link: " + year_qtr_url)
        
        file_content = requests.get(yr_qtr_url_index)
        if (file_content.status_code == 200):  # if status code = 500: a "JSONDecodeError  line 1 column 1 (char 0)"

            decoded_content = file_content.json()

            print('-'*100)
            print('Pulling files')

            # for each file in the directory items list, print the file type and file href.
            # AGAIN DOING A SUBSET
            for file in decoded_content['directory']['item']:
                if file['name'].startswith('master'):
                    file_urls.append( make_url(year_qtr_url, [ file['name']]))
                    print("File URL Link: ",  make_url(year_qtr_url, [ file['name']]))

----------------------------------------------------------------------------------------------------
Pulling url for Year: 2018
Year + Quarter URL Link: https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1
----------------------------------------------------------------------------------------------------
Pulling files
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180102.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180103.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180104.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180105.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180108.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180109.idx
File URL Link:  https://www.sec.gov/Archives/edgar/daily-index/2018/QTR1/master.20180110.idx
File URL Link:  https://

# Read all SEC Master IDX into Pandas and pick only pertinent filing from one company to output

In [7]:
txt_url = []
archive_url = []
for file_url in file_urls:

    byte_data = requests.get(file_url).content # make request for master file
    start_index = byte_data.find(b'CIK') # find the starting position of the columns (CIK is the first column as you can see from above)
    data = byte_data[start_index:].decode('utf-8') # create a filtered data set and decode the bytestring
    str_object = StringIO(data)


    df = pd.read_csv(str_object, delimiter='|', names = ['CIK', 'CompanyName', 'FormType', 'DateFiled', 'FileName'])#.drop(0, inplace = True) 
    df['FileURL'] = make_url(archive_url, df.FileName )
    df.convert_dtypes
    CUSIP = str('1392694')
    for each in df[df.FormType.str.contains('10-Q', na = False) & df.CIK.astype(str).str.contains(CUSIP)].FileName:

        URL_file = 'http://www.sec.gov/Archives/'+ str(each)
        txt_url.append(URL_file)

        # Create a url that takes us to the Detail filing landing page
        file_url_adj = URL_file.split('.txt')
        file_url_archive = file_url_adj[0] + '-index.html'

        print(URL_file)
        print('The Filing Detail can be found here: {}'.format( file_url_archive))

        # Create a url that will take us to the archive folder
        archives_url = URL_file.replace('.txt','').replace('-','')
        archive_url.append(archives_url)
        print('The Archive Folder can be found here: {}'.format(archives_url))

        # Create a url that will take us the Company filing Archive
        company_url = URL_file.rpartition('/')
        print('The Company Archive Folder can be found here: {}'.format(company_url[0]))
        print('-'*100)

http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000030.txt
The Filing Detail can be found here: http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000030-index.html
The Archive Folder can be found here: http://www.sec.gov/Archives/edgar/data/1392694/000149315218000030
The Company Archive Folder can be found here: http://www.sec.gov/Archives/edgar/data/1392694
----------------------------------------------------------------------------------------------------
http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000123.txt
The Filing Detail can be found here: http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000123-index.html
The Archive Folder can be found here: http://www.sec.gov/Archives/edgar/data/1392694/000149315218000123
The Company Archive Folder can be found here: http://www.sec.gov/Archives/edgar/data/1392694
----------------------------------------------------------------------------------------------------
http://www.sec.gov/Archives/

### User headers to bupass 403 Error

In [8]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
} 

# New Filings have the financials in a nice excle file. We can download and save that

In [9]:
for archives in archive_url:
    parsed = urllib.parse.urlsplit(archives)
    text = "{}{}".format(parsed.path.split("/")[-1], parsed.query)
    print(text)

for archive in archive_url:
    excels = archive + '/Financial_Report.xlsx'
    print(excels)   

000149315218000030
000149315218000123
000149315218000304
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000030/Financial_Report.xlsx
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000123/Financial_Report.xlsx
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/Financial_Report.xlsx


### To download the excel files

In [10]:
for archive in archive_url:
    parsed = urllib.parse.urlsplit(archive)
    text = "{}{}".format(parsed.path.split("/")[-1], parsed.query)
    
    excels = archive + '/Financial_Report.xlsx'
    r = requests.get(excels, headers = header).content
    
    writer = pd.ExcelWriter(r'file{}.xlsx'.format(text) , engine='xlsxwriter')
    xlsx = pd.ExcelFile(r)
    for sheet in xlsx.sheet_names:
        df = xlsx.parse(sheet_name = sheet, index_col=0)
        df.to_excel(writer, sheet_name = sheet)    
    writer.save()
    writer.close()

C:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# If the file does not have 'financial_report.xlsx' then we go through the documents check out which tables there are, specfy the table that we want and then download those

In [11]:
txt_url

['http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000030.txt',
 'http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000123.txt',
 'http://www.sec.gov/Archives/edgar/data/1392694/0001493152-18-000304.txt']

In [12]:
for results in txt_url:
    r = requests.get(results, headers = header).content
    dfs = pd.read_html(r)
    print(f'Total tables: {len(dfs)}')

Total tables: 477
Total tables: 476
Total tables: 493


## Make a list of all the tables by their names, to see which one is the one we want

In [17]:
data = []
for results in txt_url:

    content = requests.get(results).content
    soup = BeautifulSoup(content, 'lxml')
    accession_number = soup.find('sec-document')

    for count, report in enumerate(soup.find_all('report')):
        if report.htmlfilename is None:
            pass
        else:
            report_dict = {}

            report_dict['file_url'] = archives_url + '/' + report.htmlfilename.text
            report_dict['name_short'] = report.shortname.text
            report_dict['name_long'] = report.longname.text
      
            data.append(report_dict)

            print('-'*100)
            print(report.longname.text)
            print(report.shortname.text)
            print (archives_url + '/' + report.htmlfilename.text)

----------------------------------------------------------------------------------------------------
00000001 - Document - Document and Entity Information
Document and Entity Information
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R1.htm
----------------------------------------------------------------------------------------------------
00000002 - Statement - Consolidated Balance Sheets (Unaudited)
Consolidated Balance Sheets (Unaudited)
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R2.htm
----------------------------------------------------------------------------------------------------
00000003 - Statement - Consolidated Balance Sheets (Unaudited) (Parenthetical)
Consolidated Balance Sheets (Unaudited) (Parenthetical)
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R3.htm
----------------------------------------------------------------------------------------------------
00000004 - Statement - Consolidated Statements of Ope

## get the specific tables that we want and get the address

In [18]:
item1 = r'Consolidated Balance Sheets (Unaudited)'
item2 = r"Consolidated Statements of Operations (Unaudited)"
item3 = r"Consolidated Statements of Cash Flows (Unaudited)"
item4 = r"Stockholder's Equity"
    
report_list = [item1, item2, item3, item4]

In [19]:
tables = []
for data_points in data:
    for items in report_list:
        if data_points['name_short'] == items:
            tables.append(data_points['file_url'])
            print (data_points['file_url'])
tables            

http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R2.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R4.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R5.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R12.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R2.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R4.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R5.htm
http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R12.htm


['http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R2.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R4.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R5.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R12.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R2.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R4.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R5.htm',
 'http://www.sec.gov/Archives/edgar/data/1392694/000149315218000304/R12.htm']

###  To download the tables

In [20]:
for results in tables:
    parsed = urllib.parse.urlsplit(results)
    text = "{}{}".format(parsed.path.split("/")[-2:], parsed.query)
    
    r = requests.get(results, headers = header).content
    dfs = pd.read_html(r)
    xlWriter = pd.ExcelWriter('tables{}.xlsx'.format(text), engine='xlsxwriter')
    for i, df in enumerate(dfs):
        
        df.to_excel(xlWriter, sheet_name='Sheet{}'.format(i+1))
    xlWriter.save()